# Tinkoff Seminar
## LM, Seq2seq

На этом семинаре разберем с вами

1. Процесс декодирования в генеративных авторегрессионных моделях
2. Как работать с моделями huggingface.transformers
3. Promt engineering
4. Обучение и инференс GPT2
5. Инференс MT моделей

Семинар создан [@fursov](https://t.me/fursov)

In [ ]:
!nvidia-smi -i 3

In [ ]:
# если у вас несколько ГПУ на выбор, можете выбрать одну из них с помощью переменной окружения

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
device = torch.device('cuda')

## GPT2

В этом разделе разберемся, как работать с GPT2 в библиотеке transformers

In [ ]:
MODEL_NAME = 'sberbank-ai/rugpt3medium_based_on_gpt2'

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

### Работа с токенизатором в transformers

Как из текста сделать вход для модели?

GPT2 требует на вход 

1. input_ids -- это айдишники токенов
2. attention_mask -- это указатель, куда мы можем делать атеншн (мы не хотим делать атеншн на специальные токены)

In [ ]:
text = 'всем привет'

In [ ]:
inputs = tokenizer(text)
inputs

In [ ]:
# Оказывается, перед нами не dict!
type(inputs)

Мы можем посмотреть, какие интересные методы есть у этого объекта в [документации](https://huggingface.co/transformers/main_classes/tokenizer.html#transformers.BatchEncoding.to)


Оказывается, есть метод `.to(device)`, который сильно упрощает жизнь!



In [ ]:
# мы хотим получить торч-тензоры
inputs = tokenizer(text, return_tensors='pt')
inputs

In [ ]:
inputs.to(device)

In [ ]:
# [batch_size, num_tokens]
inputs['input_ids'].shape

In [ ]:
# GPT2 учится без падингов. все тексты конкатинируются вместе

tokenizer.pad_token

In [ ]:
# но есть специальный токен конца текста

tokenizer.eos_token

С помощью токенайзера мы можем, как энкодить, так и декодить последовательность айдишников

In [ ]:
tokenizer.decode([ 275, 1515, 6129, 123])

### Процесс инференса GPT2

In [ ]:
# inplace операция!

inputs.to(device)

In [ ]:
# действительно

inputs

In [ ]:
# вызываем .forward

out = model(**inputs)

In [ ]:
type(out)

In [ ]:
out.keys()

По логитам можем предсказать следующее слово

In [ ]:
out['logits'].shape

In [ ]:
# берем наиболее вероятный токен

next_token_id = out['logits'][:, -1, :].argmax()

In [ ]:
next_token_id

In [ ]:
sentence_ids = inputs['input_ids'].cpu().numpy().tolist()[0] + [next_token_id.item()]

In [ ]:
sentence_ids

Модель решила поставить знак ударения

In [ ]:
tokenizer.decode(sentence_ids)

Соответственно, мы можем повторить операцию и получить следующий наиболее вероятный токен

In [ ]:
out = model(torch.tensor([sentence_ids], device=device))
next_token_id = out['logits'][:, -1, :].argmax()
sentence_ids = sentence_ids + [next_token_id.item()]

In [ ]:
tokenizer.decode(sentence_ids)

## Decoding

Давайте попробуем сами реализовать top-k, top-p сэмплирвование с температурой. Но для начала попробуем просэмплировать без top-k

In [ ]:
inputs = tokenizer('всем привет', return_tensors='pt').to(device)
input_ids_list = inputs['input_ids'][0].cpu().numpy().tolist()

out = model(**inputs)
logits = out.logits
next_token_logits = logits[:, -1, :]

In [ ]:
next_token_logits.shape

In [ ]:
def sample_from_logits(next_token_logits, num_samples=10):
    # получаем вероятности с помощью софтмакса
    probs = torch.softmax(next_token_logits, dim=-1)
    next_tokens = torch.multinomial(probs, num_samples=num_samples, replacement=True)

    for next_token in next_tokens[0]:
        decoded = tokenizer.decode(input_ids_list + [next_token.item()])
        print(decoded)

In [ ]:
# сэмплируем из мультиноминального распределения

sample_from_logits(next_token_logits)

Примеры получились более-менее. А что если добавить температуру?

In [ ]:
temperature = 2.0

sample_from_logits(next_token_logits / temperature)

Температура оказалось слишком высокой, так как некоторые примеры странные

In [ ]:
temperature = 0.5

sample_from_logits(next_token_logits / temperature)

In [ ]:
temperature = 1.1

sample_from_logits(next_token_logits / temperature)

Температура ниже 0.5 ведет к более консервативным генерациям. Пробуем занулить вероятности токенов с помощью top-k

In [ ]:
top_k = 5

def apply_top_k(scores, top_k=top_k):
    indices_to_remove = scores < torch.topk(scores, top_k)[0][..., -1, None]
    scores = scores.masked_fill(indices_to_remove, -float("Inf"))
    return scores

Мы получаем всего 5 уникальных примеров

In [ ]:
sample_from_logits(
    apply_top_k(next_token_logits), num_samples=20
)

Соответственно, мы теперь можем даже комбинировать top-k и температуру. Подбирать гиперпараметры декодирования нужно очень внимательно

In [ ]:
sample_from_logits(
    apply_top_k(next_token_logits / 2.0, top_k=10)
)

Хотя температура высокая, но top-k не дает генерировать совсем неправдоподобные варианты

## GenerationMixin

Все генеративные модели в библиотеке **transformers** имеют метод generate, что упрощает нам генерацию

In [ ]:
decoded = model.generate(**inputs)

In [ ]:
decoded

In [ ]:
tokenizer.decode(decoded[0])

Для понимания, советую включать **return_dict_in_generate=True**

In [ ]:
decoded = model.generate(**inputs, return_dict_in_generate=True)

In [ ]:
decoded

In [ ]:
decoded.sequences

In [ ]:
type(decoded)

Теперь вместо последовательности айдишников мы по типу объекта можем понять, какой алгоритм декодирования был применен.

Соответственно, можно поиграть со всеми основными типами декодирования.

In [ ]:
# бим серч

decoded = model.generate(**inputs, num_beams=10, return_dict_in_generate=True)
print(type(decoded))
tokenizer.decode(decoded.sequences[0])

In [ ]:
# сэмплирование

decoded = model.generate(**inputs, do_sample=True, return_dict_in_generate=True)
print(type(decoded))
tokenizer.decode(decoded.sequences[0])

In [ ]:
# сэмплирование c температурой

decoded = model.generate(**inputs, do_sample=True, temperature=0.5, return_dict_in_generate=True)
print(type(decoded))
tokenizer.decode(decoded.sequences[0])

### Проблемы с повторами

Что делать если модель начала повторяться?

In [ ]:
text = 'всем привет всем привет всем привет всем привет всем привет всем привет всем привет'
inputs = tokenizer(text, return_tensors='pt')
inputs.to(device)

In [ ]:
# модель продолжает генерировать "всем привет"
generated = model.generate(**inputs)
tokenizer.decode(generated[0])

In [ ]:
# мы можем перевзвесить вероятности для токенов, которые мы уже видели
# идея отсюда: https://arxiv.org/pdf/1909.05858.pdf

generated = model.generate(**inputs, repetition_penalty=4.0)
tokenizer.decode(generated[0])

## Prompt Engineering

GPT2 модель достаточно сильная даже без дообучения, чтобы решать разные задачи

### Рекомендации

In [ ]:
inputs = tokenizer('Самый лучший фильм это —', return_tensors='pt')
inputs.to(device)

In [ ]:
generated = model.generate(**inputs, num_beams=4)
tokenizer.batch_decode(generated)

In [ ]:
# модель может посоветовать фильм, который стоит посмотреть

generated = model.generate(**inputs, do_sample=True, top_p=0.7, num_return_sequences=4)
tokenizer.batch_decode(generated)

### Диалоговый агент

In [ ]:
text = """— Привет
— Здраствуй
— Как дела?
"""

inputs = tokenizer(text, return_tensors='pt')
inputs.to(device)

In [ ]:
# модель умеет продолжать диаплог

generated = model.generate(**inputs, do_sample=True, top_p=1.7, num_return_sequences=4)
tokenizer.batch_decode(generated)

### [Your idea here]

Попробуйте придумать, где еще можно применить GPT!

Вдохновиться можно на примерах отсюда https://github.com/elyase/awesome-gpt3

### Fine-tuning

Если вам понадобиться дообучить GPT2 под свои задачи (например, в Тинькофф можно дообучить на данных общения операторов с клиентами), то лучшим способом будет воспользоваться скриптами из huggingface: https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling


А если вы вдруг заходите затюнить очень большую модель, то лучший способ распараллелить обучение у NVIDIA: https://github.com/NVIDIA/Megatron-LM

# Обучаем GPT2 генерировать стихи

Мы собрали датасет четверостиший. Попробуем на нем обучить GPT2 генерировать стихи. Нам из датасета нужно сделать файл, на котором мы сможем обучиться.

In [ ]:
import json

In [ ]:
data = []

with open('data/data.json') as f:
    for line in f:
        data.append(json.loads(line.strip()))

In [ ]:
len(data)

In [ ]:
data[128]

Нам нужно подготовить .txt файл, где на каждой строчке будет стих. Для улучшения качества обучения мы каждую строку будем разделять специальным токеном **@@LINE_i@@**, чтобы модель различала строки между собой и могла проще предлагать рифмы.

In [ ]:
data_txt = []

for example in data:
    a, b, c, d = example
    example_txt = f'@@LINE_0@@ {a} @@LINE_1@@ {b} @@LINE_2@@ {c} @@LINE_3@@ {d}'
    data_txt.append(example_txt)

In [ ]:
data_txt[128]

In [ ]:
with open('data/data.txt', 'w') as w:
    for ex in data_txt:
        w.write(f'{ex}\n')

Мы хотим, чтобы токенизатор знал о новых специальных токенов, поэтому добавим их в словарь. (Во время обучения **в матрицу эмбедингов** мы также добавим эти новые токены)

In [ ]:
! mkdir -p data/new_tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens(['@@LINE_0@@', '@@LINE_1@@', '@@LINE_2@@', '@@LINE_3@@'], special_tokens=True)

In [ ]:
tokenizer.save_pretrained('data/new_tokenizer')

### Скрипт обучения на одной gpu выглядит следующим образом

(Пробуем запускать в терминале)

```bash
CUDA_VISIBLE_DEVICES=1 python train_gpt.py \
  --model_name_or_path "sberbank-ai/rugpt3large_based_on_gpt2" \
  --tokenizer_name "data/new_tokenizer" \
  --model_type "gpt2" \
  --train_file "data/data.txt" \
  --validation_file "data/data.txt" \
  --cache_dir "data/cache" \
  --output_dir "poem_logs" \
  --per_device_train_batch_size 3 \
  --per_device_eval_batch_size 3 \
  --save_total_limit 3 \
  --save_strategy "steps" \
  --eval_steps 500 \
  --save_steps 500 \
  --preprocessing_num_workers 4 \
  --num_train_epochs 3 \
  --block_size 512
```

После обучения пробуем сгенерировать стихи

In [ ]:
!ls logs/checkpoint-340000

In [ ]:
checkpoint_dir = 'logs/checkpoint-340000/'

model = GPT2LMHeadModel.from_pretrained(checkpoint_dir).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_dir)

## Процесс генерации

Теперь мы можем подавать на вход модели, например, первую строку стиха (или первые две) и смотреть, какое продолжение выдаст модель

In [ ]:
def print_from_indexes(indexes):
    decoded = tokenizer.decode(indexes[0]).split('@@')
    for num, i in enumerate([2, 4, 6, 8]):
        try:
            line = decoded[i].strip()
            if line:
                print(line)
            else:
                print(f'[Line {num + 1} is not generated yet]')
        except IndexError:
            print(f'[Line {num + 1} is not generated yet]')

In [ ]:
# запомним айдишники специальных токенов
vocab = tokenizer.get_vocab()

In [ ]:
id0, id1, id2, id3 = vocab['@@LINE_0@@'], vocab['@@LINE_1@@'], vocab['@@LINE_2@@'], vocab['@@LINE_3@@']

In [ ]:
inputs = tokenizer('@@LINE_0@@ У лукоморья дуб зелёный @@LINE_1@@', return_tensors='pt')
inputs.to(device)

In [ ]:
# останавливаем генерацию, как только увидим @@LINE_2@@

generated = model.generate(**inputs, num_beams=5, eos_token_id=id2)

print_from_indexes(generated)

In [ ]:
# добавляем рандома
generated = model.generate(**inputs, do_sample=True, temperature=1.1, eos_token_id=id2)

print_from_indexes(generated)

In [ ]:
# генерим полный стих
generated = model.generate(**inputs, do_sample=True, temperature=1.1, eos_token_id=id0, max_length=100)

print_from_indexes(generated)

In [ ]:
# пробуем другую затравку

inputs = tokenizer('@@LINE_0@@ Тинькофф заходит в дом @@LINE_1@@', return_tensors='pt')
inputs.to(device)
generated = model.generate(**inputs, num_beams=5, eos_token_id=id0, repetition_penalty=4.0, max_length=100)

print_from_indexes(generated)

# Seq2seq на примере MT

Работа с Seq2seq моделями в `transformers` очень похожа на работу с LM моделями. Давайте попробуем с помощью opensource модели с https://huggingface.co/models перевести какой-нибудь текст.

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

In [ ]:
MT_MODEL_NAME = "Helsinki-NLP/opus-mt-en-ru"

In [ ]:
tokenizer = MarianTokenizer.from_pretrained(MT_MODEL_NAME)
model = MarianMTModel.from_pretrained(MT_MODEL_NAME)

In [ ]:
# перед нами типичная transformer-like encoder-decoder архитектура

model.eval().to(device)

In [ ]:
en_texts = [
    "I'm gonna make him an offer he can't refuse.",
    "Here's looking at you, kid.",
    "Frankly, my dear, I don't give a damn.",
    "Why so serious?",
    "Say hello to my li’l friend!",
    "Life is like a box of chocolates"
]

В этот раз мы будем работать сразу с батчом текстов. Поэтому нам нужно включить `padding` (увеличение длины последовательности с помощью pad token), `truncation` (обрезание длины текста до максимально допустимой длины)

In [ ]:
batch_text_inputs = tokenizer(
    en_texts,
    max_length=256,
    return_tensors='pt',
    padding=True,
    truncation=True
)

batch_text_inputs = batch_text_inputs.to(device)

In [ ]:
batch_text_inputs

In [ ]:
batch_text_inputs['input_ids'].shape

In [ ]:
# точно такой же метод для генерации!
# хорошей идеей будет всегда включать beam search

output = model.generate(**batch_text_inputs, num_beams=5)

In [ ]:
translations = tokenizer.batch_decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
translations

Теперь мы можем посчитать **BLEU** скор, приняв перевод из гугл-переводчика за правильный

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
google_translations = [
    "Я сделаю ему предложение, от которого он не сможет отказаться.",
    "Тут присматривают за тобой, дитя.",
    "Честно говоря, моя дорогая, мне наплевать.",
    "Почему ты такой серьезный?",
    "Передай привет моему маленькому другу!",
    "Жизнь похожа на коробку конфет"
]

In [ ]:
bleus = []

for ru_true, ru_pred in zip(google_translations, translations):
    bleu = sentence_bleu([ru_pred.lower()], ru_true.lower())
    bleus.append(bleu)

In [ ]:
# Судя по этой метрике, качество перевода не очень хорошее, хотя перевод модели мне нравится

mean_bleu = sum(bleus) / len(bleus)
print(f'BLEU = {mean_bleu:.2f}')